# dPb concentrations across the domain for a single depth level

In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import pandas as pd
import xarray as xr

import gsw
import matplotlib.backends.backend_pdf

%matplotlib inline

#### Parameters:

In [4]:
# domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179
isize = imax - imin
jsize = jmax - jmin

# model results:
year = 2015
results_folder1 = f'/data/brogalla/run_storage/Pb-tuning-202302/Pb-reference-{year}_20230221/'
# results_folder1 = f'/data/brogalla/run_storage/Pb-tuning-202302/Pb-nosedice-{year}_20230221/'

# colors:
c_masked = '#eaeae9'

#### Load files:

In [5]:
mfile        = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
ANHA12_lon   = mfile['nav_lon'].values
ANHA12_lat   = mfile['nav_lat'].values
tmask        = mfile['tmask'].values
tmask_sub    = mfile['tmask'].values[0,:,imin:imax,jmin:jmax]

mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_bathy   = mesh['hdept'].values[0]
e3t          = mesh['e3t_0'].values[0,:,imin:imax,jmin:jmax]
land_mask    = np.ma.masked_where((tmask[0,:,:,:] > 0.1) , tmask[0,:,:,:]) 
e3t_masked   = np.ma.masked_where((tmask_sub[:,:,:] < 0.1), e3t)

In [6]:
month  = 8
results     = xr.open_dataset(f'{results_folder1}ANHA12_EXH006_1m_{year}0101_{year}1231_comp_{year}{month:02}-{year}{month:02}.nc')
lons_model  = results['nav_lon'].values
lats_model  = results['nav_lat'].values
depth_model = results['deptht'].values
dissolpb    = results['dissolpb'].values[0,:,:,:]*1e12 # pM units

In [7]:
# --- GEOTRACES observations: ---
GTR           = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/IDP2021_GEOTRACES_IDP2021_Seawater_Discrete_Sample_Data_v1_kYw7mpQ2.nc')
GTR_cruises   = GTR['metavar1']
GTR_lons      = GTR['longitude'].values              # degrees east
GTR_lats      = GTR['latitude'].values               # degrees north
GTR_pres      = GTR['var1'].values           # Pressure from CTD sensor [dbar]
GTR_depths    = GTR['var2'].values                   # Depth below sea surface calculated from pressure [m]
GTR_temp      = GTR['var3'].values                   # Temperature from CTD sensor in the ITS-90 convention. [C]
GTR_sal       = GTR['var4'].values                   # Practical salinity from CTD sensor on the PSS-1978 scale
GTR_names     = GTR['metavar2'].astype('str').values # Station names

# --- Dissolved Pb concentrations: ---
GTR_qc         = GTR['var16_qc'].values # flag = 51, 52 are probably bad and bad 
GTR_dPb_pmolkg = GTR['var16'].values
GTR_dPb_pmolkg[(GTR_qc == 51) | (GTR_qc == 52)] = np.nan # exclude quality control flagged values

# --- Convert pmol / kg to pmol / L --- (pmol/kg * kg/m3 * 1 m3 / 1000 L)
GTR_pressure  = np.ma.masked_where(GTR_pres < -1e8, GTR_pres)
# practical salinity --> absolute salinity (g/kg)
GTR_lons_tile = np.tile(GTR_lons, (345,1)).transpose()
GTR_lats_tile = np.tile(GTR_lats, (345,1)).transpose()
GTR_sal_abs   = gsw.conversions.SA_from_SP(GTR_sal, GTR_pressure, GTR_lons_tile, GTR_lats_tile)
# calculate density: uses in situ temperature, absolute salinity, sea pressure 
GTR_density   = gsw.density.rho_t_exact(GTR_sal_abs, GTR_temp, GTR_pressure) # kg/m3
GTR_dPb_pmolL = GTR_dPb_pmolkg*GTR_density*1e-3
GTR_dPb_pmolL[GTR_dPb_pmolL < 0] = np.nan # can't have negative concentrations

#### Functions:

In [8]:
def plot_field(variable, layer, vmin=0, vmax=35, obs=False, choice_map='PC'):
    cmap1 = cmocean.cm.matter
    cmap1.set_bad(c_masked, 1)   
    
    if choice_map=='PC':
        fig, ax, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/Pb-PC.pickle','rb'))
    elif choice_map=='NS':
        fig, ax, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/Pb-Nares.pickle','rb'))
    elif choice_map=='BB':
        fig, ax, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/Pb-Baffin.pickle','rb'))

    x, y = proj1(lons_model, lats_model) 
    
    ax.set_facecolor(c_masked)
    plot_variable = np.ma.masked_where(tmask_sub[layer,:,:] < 0.1, variable[layer,:,:])
    pcolor  = ax.pcolormesh(x, y, plot_variable, cmap=cmap1, vmin=vmin, vmax=vmax, rasterized=True)
    contour = ax.contourf(np.zeros_like(x), np.zeros_like(y), np.zeros_like(plot_variable), np.arange(vmin,vmax+1,1),\
                          cmap=cmap1, vmin=vmin, vmax=vmax, extend='both')

    if obs: #plot circles with 2015 GEOTRACES observations
        GTR_lons_ma = np.ma.masked_where((GTR_lons < 200) & (GTR_lats > 85) | (GTR_lats < 65) & (GTR_lons > 300) | \
                                         (GTR_lats < 85) & (GTR_lons > 0) & (GTR_lons < 150), GTR_lons)
        GTR_lats_ma = np.ma.masked_where((GTR_lons < 200) & (GTR_lats > 85) | (GTR_lats < 65) & (GTR_lons > 300) | \
                                         (GTR_lats < 85) & (GTR_lons > 0) & (GTR_lons < 150), GTR_lats)
        x_GTR, y_GTR = proj1(GTR_lons_ma, GTR_lats_ma)

        # filter based on depth:
        if depth_model[layer] < 200: # in shallow areas use only observations that are within 15 m from layer
            depth_mask     = (GTR_depths > depth_model[layer] - 10) & (GTR_depths < depth_model[layer] + 10)
        else:  # in deeper waters use observations that are within 50 m from the layer
            depth_mask     = (GTR_depths > depth_model[layer] - 25) & (GTR_depths < depth_model[layer] + 25)

        GTR_dPb_masked = np.ma.masked_where(~depth_mask, GTR_dPb_pmolL) # calculate mean over observations within this range
        GTR_dPb_slice  = np.nanmean(GTR_dPb_masked, axis=1)
        ax.scatter(x_GTR, y_GTR, c=GTR_dPb_slice, s=80, zorder=3, edgecolor='w', marker='.', 
                   linewidth=0.5, vmin=vmin, vmax=vmax, cmap=cmap1, label='2015 GEOTRACES GN02, GN03')

        ax.set_title(f'Depth: {depth_model[layer]:.0f} m', fontsize=7)
            
    
    cbaxes = fig.add_axes([0.03, 0.4, 0.02, 0.18]);
    CBar   = plt.colorbar(contour, ax=ax, cax=cbaxes); 
    CBar.set_label('Dissolved Pb [pM]', fontsize=6)
    CBar.ax.tick_params(axis='y', length=0, labelsize=6)
            
    return fig

#### Figures of surface concentrations:

In [ ]:
depth_levels = np.arange(0,30,1)
fig = {}
for depth in depth_levels:
    fig[f'z{depth}'] = plot_field(dissolpb, depth, vmin=0, vmax=15, obs=True, choice_map='PC')   
    
pdf = matplotlib.backends.backend_pdf.PdfPages("/ocean/brogalla/GEOTRACES/figures/Pb-Parry-Channel-2015-ref.pdf")
for depth in depth_levels:
    pdf.savefig(fig[f'z{depth}'])
pdf.close()

In [ ]:
depth_levels = np.arange(0,31,1)
fig = {}
for depth in depth_levels:
    fig[f'z{depth}'] = plot_field(dissolpb, depth, vmin=0, vmax=15, obs=True, choice_map='NS')    

pdf = matplotlib.backends.backend_pdf.PdfPages("/ocean/brogalla/GEOTRACES/figures/Pb-Nares-Strait-2015-ref.pdf")
for depth in depth_levels:
    pdf.savefig(fig[f'z{depth}'])
pdf.close()

In [ ]:
depth_levels = np.arange(0,42,1)
fig = {}
for depth in depth_levels:
    fig[f'z{depth}'] = plot_field(dissolpb, depth, vmin=0, vmax=15, obs=True, choice_map='BB')    

pdf = matplotlib.backends.backend_pdf.PdfPages("/ocean/brogalla/GEOTRACES/figures/Pb-Baffin-Bay-2015-ref.pdf")
for depth in depth_levels:
    pdf.savefig(fig[f'z{depth}'])
pdf.close()